In [1]:
from feature_extractor import EEGCSVreader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata


# メイン処理
subject = 'subject1'
data_dir = f'pump_split_modeling/train/{subject}'  # データが格納されているディレクトリのパスを指定

# 特徴量抽出
csvReader = EEGCSVreader(data_dir)
try:
    X, y = csvReader.transform(None)
except ValueError as e:
    print(f"Error: {str(e)}")
    print("Please check your data directory and ensure all files have the correct format.")
    exit(1)


In [2]:
channel_pos = pd.read_csv('channel_pos.csv', index_col=0)
channel_pos = channel_pos.apply(lambda x: round((x-0.5)*20).astype(int))

# trial毎に5枚画像を作る，(trialx5)x18x18のndarrayを作る
all_images = []
for trial in X:
    # trialを５分割
    split_all = np.array_split(trial, 5)

    for split in split_all:
        mean = split.mean(axis=0)

        pos = channel_pos.T.to_numpy()
        val = mean.values
        grid_x, grid_y = np.mgrid[-9:10, -9:10]

        filled = griddata(pos, val, (grid_x, grid_y), method='cubic')
        filled = filled[:-1,1:]     # 18x18
        all_images.append(filled)

all_images = np.array(all_images)

print(all_images.shape)

# plt.imshow(filled, cmap='bwr', vmin=-5, vmax=5)
# plt.colorbar()
# plt.show()


/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated a

(1600, 18, 18)


/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated a

### データ準備

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

all_images = all_images.astype(np.float32)

# データをPyTorchのテンソルに変換
X_tensor = torch.tensor(all_images).unsqueeze(1)  # (N, 1, 19, 19) に変換 (チャネル数を1にする)

# DataLoaderを作成
batch_size = 32
dataset = TensorDataset(X_tensor, X_tensor)  # 入力と出力が同じ（自己符号化器のため）
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


### ネットワーク定義

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder: 入力を潜在空間に圧縮
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # (1, 18, 18) -> (16, 18, 18)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # (16, 18, 18) -> (16, 9, 9)
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # (16, 9, 9) -> (32, 9, 9)
            nn.ReLU(),
            nn.MaxPool2d(2, 2)   # (32, 9, 9) -> (32, 4, 4)
        )
        
        # Decoder: 潜在空間から元の次元に復元
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, output_padding=0),  # (32, 4, 4) -> (16, 9, 9)
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, output_padding=1),   # (16, 9, 9) -> (1, 18, 18)
            nn.Sigmoid()  # 出力を0~1にスケーリング
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


### 学習設定

In [7]:
# モデルのインスタンス化
model = Autoencoder()

# 損失関数と最適化手法を定義
criterion = nn.MSELoss()  # 入力と出力の差を最小化
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習ループ
n_epochs = 50
for epoch in range(n_epochs):
    running_loss = 0.0
    for data in dataloader:
        inputs, _ = data
        optimizer.zero_grad()
        
        # 順伝播
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        
        # 逆伝播とパラメータ更新
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {running_loss/len(dataloader):.4f}')

/Users/onumayosuke/Documents/dev/signateEEG/signateEEG/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([32, 1, 18, 18])) that is different to the input size (torch.Size([32, 1, 20, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (20) must match the size of tensor b (18) at non-singleton dimension 3

### 潜在空間でkmeans

In [ ]:
# エンコーダから潜在空間を取得
def get_latent_space(model, dataloader):
    latent_space = []
    model.eval()  # モデルを推論モードに切り替え
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            encoded = model.encoder(inputs)
            latent_space.append(encoded.view(encoded.size(0), -1).cpu().numpy())  # Flattenして保存
    return np.concatenate(latent_space, axis=0)

# 潜在空間を取得
latent_space = get_latent_space(model, dataloader)

# k-meansでクラスタリング
from sklearn.cluster import KMeans

n_clusters = 3  # クラスターの数はデータに合わせて調整
kmeans = KMeans(n_clusters=n_clusters)
clusters = kmeans.fit_predict(latent_space)

print("Clusters:", clusters)
